In [1]:
raw_tr = pd.read_json("./raw_data/train.json")
raw_te = pd.read_json("./raw_data/val.json")

In [2]:
from sklearn.preprocessing import MultiLabelBinarizer

raw_tr_tags_mlb = MultiLabelBinarizer(sparse_output=True)

In [3]:
raw_tr_tags = list(raw_tr.tags)
tag_df = raw_tr_tags_mlb.fit_transform(raw_tr_tags)
tag_classes = raw_tr_tags_mlb.classes_

In [4]:
tag_classes

array(['', '00', '007', ..., '힛뎀포크', '힛뎀폭', '힛뎀폭스'], dtype=object)

In [5]:
raw_tr_songs_mlb = MultiLabelBinarizer(sparse_output=True)
raw_tr_songs = list(raw_tr.songs)
songs_df = raw_tr_songs_mlb.fit_transform(raw_tr_songs)
songs_classes = raw_tr_songs_mlb.classes_

In [6]:
songs_classes

array([     0,      3,      4, ..., 707986, 707987, 707988])

In [7]:
data = sp.sparse.hstack([songs_df, tag_df])

In [8]:
data

<115071x644302 sparse matrix of type '<class 'numpy.int64'>'
	with 5762202 stored elements in COOrdinate format>

In [9]:
df = pd.DataFrame(data.todense())

In [ ]:
del data

In [8]:
# 변수 삭제
del raw_te
del raw_tr
del raw_tr_songs
del raw_tr_tags

In [11]:
df.shape

(115071, 644302)

In [19]:
# from keras.datasets import mnist
from keras.layers import Input, Dense
from keras.models import Model
import keras.backend.tensorflow_backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint


# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# configure
encoding_dim = 32
input_data = Input(shape=(644302,))


# layers
encoded = Dense(encoding_dim, activation='relu')(input_data)
decoded = Dense(644302, activation='sigmoid')(encoded)


with K.tf_ops.device('/device:GPU:0'):
    # Models
    autoencoder = Model(input_data, decoded) # autoencoder
    encoder = Model(input_data, encoded) # encoder
    encoded_input = Input(shape=(encoding_dim,))
    decoder_layer = autoencoder.layers[-1]
    decoder = Model(encoded_input, decoder_layer(encoded_input)) # decoder
    model_dir = './model'


#     model_path = model_dir + '/multi_img_classification'
#     checkpoint = ModelCheckpoint(filepath=model_path , monitor='recall', verbose=1, save_weights_only=True,     period=5)
#     early_stopping = EarlyStopping(monitor='loss', patience=5)

In [22]:
# result viewer
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def recall(y_true, y_pred):
    y_true_yn = K.round(K.clip(y_true, 0, 1))
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))

    count_true_positive = K.sum(y_true_yn * y_pred_yn)
    count_true_positive_false_negative = K.sum(y_true_yn)
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    return recall

# train autoencoder
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=[rmse])
autoencoder.fit(data, data,
                epochs=10,
                batch_size=256,
                shuffle=True, validation_split=0.2, callbacks=[checkpoint,early_stopping])

TypeError: 'coo_matrix' object is not subscriptable

In [ ]:
# encoding result
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

# classification train data
reducted_x_train = encoder.predict(x_train)
reducted_x_test = encoder.predict(x_test)

# train classifier
model=Sequential()
model.add(Dense(64, activation='relu', input_dim=encoding_dim))
model.add(layers.Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# final result
history = model.fit(reducted_x_train, y_train, epochs=10, batch_size=100, validation_split=0.2)
performance_test = model.evaluate(reducted_x_test, y_test, batch_size=100)
print('Test Loss and Accuracy ->', performance_test)

In [9]:
# Importing tensorflow
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# Importing some more libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

Instructions for updating:
non-resource variables are not supported in the long term


In [10]:
# creating train and test sets
X_train, X_test = train_test_split(data, train_size=0.8)

In [11]:
print(X_train.shape)
print(X_test.shape)

(92056, 644302)
(23015, 644302)


In [12]:
# Deciding how many nodes wach layer should have
n_nodes_inpl = 644302  
n_nodes_hl1  = 256  
n_nodes_outl = 644302  
# first hidden layer has 784*32 weights and 32 biases
hidden_1_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_inpl+1,n_nodes_hl1]))}
# first hidden layer has 784*32 weights and 32 biases
output_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1+1,n_nodes_outl])) }

In [13]:
# user with 3706 ratings goes in
input_layer = tf.placeholder('float', [None, 644302])
# add a constant node to the first layer
# it needs to have the same shape as the input layer for me to be
# able to concatinate it later

input_layer_const = tf.fill( [tf.shape(input_layer)[0], 1] ,1.0  )
input_layer_concat =  tf.concat([input_layer, input_layer_const], 1)

# multiply output of input_layer wth a weight matrix 
layer_1 = tf.nn.sigmoid(tf.matmul(input_layer_concat, hidden_1_layer_vals['weights']))

# adding one bias node to the hidden layer
layer1_const = tf.fill( [tf.shape(layer_1)[0], 1] ,1.0  )
layer_concat =  tf.concat([layer_1, layer1_const], 1)

# multiply output of hidden with a weight matrix to get final output
output_layer = tf.matmul( layer_concat,output_layer_vals['weights'])

# output_true shall have the original shape for error calculations
output_true = tf.placeholder('float', [None, 644302])

# define our cost function
meansq =    tf.reduce_mean(tf.square(output_layer - output_true))

# define our optimizer
learn_rate = 0.1   # how fast the model should learn
optimizer = tf.train.AdagradOptimizer(learn_rate).minimize(meansq)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
# initialising variables and starting the session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# defining batch size, number of epochs and learning rate
batch_size = 100  # how many images to use together for training
hm_epochs =200    # how many times to go through the entire dataset
tot_users = data.shape[0] # total number of images
tot_images = data.shape[1]

In [17]:
df = data.todense()

In [ ]:
# running the model for a 200 epochs taking 100 users in batches
# total improvement is printed out after each epoch
for epoch in range(hm_epochs):
    epoch_loss = 0    # initializing error as 0
    
    for i in range(int(tot_images/batch_size)):
        epoch_x = df[ i*batch_size : (i+1)*batch_size ]
        _, c = sess.run([optimizer, meansq],
               feed_dict={input_layer: epoch_x, 
               output_true: epoch_x})
        epoch_loss += c
        
    output_train = sess.run(output_layer, feed_dict={input_layer:X_train})
    output_test = sess.run(output_layer, feed_dict={input_layer:X_test})
        
    print('MSE train', MSE(output_train, X_train),'MSE test', MSE(output_test, X_test))      
    print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)